In [1]:
# =========================
# GOLD: desempenho_filmes
# =========================

# Instalar Delta
!pip install delta-spark==2.4.0


In [2]:
# Imports + Spark Session (Hive + Delta)
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, lower, trim, regexp_replace, udf, lit,
    countDistinct, collect_set, size, when, avg
)
from pyspark.sql.types import StringType, IntegerType, DoubleType
from pyspark.sql.window import Window
from delta import *
import unicodedata

warehouse_location = "hdfs://hdfs-nn:9000/warehouse"

spark = (
    SparkSession.builder
    .appName("Gold_Desempenho_Filmes")
    # ---- configurações Hive ----
    .config("spark.sql.warehouse.dir", warehouse_location)
    .config("hive.metastore.uris", "thrift://hive-metastore:9083")
    .config("spark.sql.catalogImplementation", "hive")
    .config("hive.metastore.warehouse.dir", warehouse_location)
    # ---- extensões Delta Lake ----
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    # ---- pacote Delta compatível com Spark 3.4.1 (Scala 2.12) ----
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0")
    .enableHiveSupport()
    .getOrCreate()
)

print("Spark iniciado com sucesso — versão:", spark.version)


Spark iniciado com sucesso — versão: 3.4.1


In [3]:
# Criar base de dados GOLD (se não existir)
spark.sql("""
    CREATE DATABASE IF NOT EXISTS gold
    LOCATION 'hdfs://hdfs-nn:9000/warehouse/gold.db'
""")
spark.sql("USE gold")
print("Base de dados 'gold' pronta.")


Base de dados 'gold' pronta.


In [4]:
# Função UDF para remover acentos (mesma lógica da Silver)
def remove_accents(text):
    if text is None:
        return None
    return ''.join(
        c for c in unicodedata.normalize('NFKD', text)
        if not unicodedata.combining(c)
    )

remove_accents_udf = udf(remove_accents, StringType())

def normalize_title_col(c):
    """
    Normalização compatível com silver.boxoffice.title e silver.adaptations.movie_title:
    - lowercase + trim
    - remover acentos
    - remover caracteres especiais (manter a-z, 0-9 e espaço)
    """
    return regexp_replace(
        remove_accents_udf(lower(trim(c))),
        r"[^a-z0-9 ]",
        ""
    )


In [5]:
# Ler tabelas Silver
adaptations = spark.table("silver.adaptations")    
boxoffice    = spark.table("silver.boxoffice")      
films_awards = spark.table("silver.films_awards")   

print("Silver loaded:")
print(" - adaptations:", adaptations.count())
print(" - boxoffice:", boxoffice.count())
print(" - films_awards:", films_awards.count())


Silver loaded:
 - adaptations: 153
 - boxoffice: 8144
 - films_awards: 5077


In [6]:
# Garantir colunas-chave consistentes

# Boxoffice (já vem com title normalizado e year int)
boxoffice_keyed = (
    boxoffice
    .select(
        col("title").alias("title_norm"),
        col("year").cast(IntegerType()).alias("year"),
        col("gross").cast(DoubleType()).alias("gross"),
        col("decade").cast(IntegerType()).alias("decade")
    )
    .dropDuplicates(["title_norm", "year"])
)

# Adaptations (já vem com movie_title normalizado)
#     Há filmes com múltiplos livros/autores -> agregamos para 1 linha por filme+ano
adaptations_agg = (
    adaptations
    .select(
        col("movie_title").alias("title_norm"),
        col("movie_year").cast(IntegerType()).alias("year"),
        col("author").alias("author_norm"),
        col("book_title").alias("book_title_norm")
    )
    .groupBy("title_norm", "year")
    .agg(
        collect_set("author_norm").alias("authors"),
        collect_set("book_title_norm").alias("book_titles"),
        countDistinct("author_norm").alias("n_authors"),
        countDistinct("book_title_norm").alias("n_books")
    )
    .withColumn("is_adaptation", lit(True))
)

# Films Awards
#     Criar "title_norm" usando normalização compatível + "year"
#     Observação: no teu silver.films_awards o ano está em "year_int"
films_awards_keyed = (
    films_awards
    .withColumn("title_norm", normalize_title_col(col("film")))
    .withColumn("year", col("year_int").cast(IntegerType()))
    .select(
        "title_norm",
        "year",
        col("oscar_nominations").cast(IntegerType()).alias("oscar_nominations"),
        col("oscar_wins").cast(IntegerType()).alias("oscar_wins"),
        col("won_any_oscar").cast("boolean").alias("won_any_oscar"),
        col("oscar_categories")
    )
    .dropDuplicates(["title_norm", "year"])
)

print("Keys prepared:")
print(" - boxoffice_keyed:", boxoffice_keyed.count())
print(" - adaptations_agg:", adaptations_agg.count())
print(" - films_awards_keyed:", films_awards_keyed.count())


Keys prepared:
 - boxoffice_keyed: 8144
 - adaptations_agg: 152
 - films_awards_keyed: 5076


In [7]:
# Construir a tabela GOLD "desempenho_filmes"
# Estratégia: base = boxoffice (porque precisamos decade + gross para Q4, Q8)
# Depois: juntar Óscares e Adaptações (left joins)

desempenho_filmes = (
    boxoffice_keyed
    .join(films_awards_keyed, on=["title_norm", "year"], how="left")
    .join(adaptations_agg,     on=["title_norm", "year"], how="left")
    # Flags e defaults
    .withColumn("won_any_oscar", when(col("won_any_oscar").isNull(), lit(False)).otherwise(col("won_any_oscar")))
    .withColumn("oscar_wins", when(col("oscar_wins").isNull(), lit(0)).otherwise(col("oscar_wins")))
    .withColumn("oscar_nominations", when(col("oscar_nominations").isNull(), lit(0)).otherwise(col("oscar_nominations")))
    .withColumn("is_adaptation", when(col("is_adaptation").isNull(), lit(False)).otherwise(col("is_adaptation")))
    .withColumn("post_2000", (col("year") >= 2000))
)

# Algumas colunas úteis para dashboard/Q8:
# - média global e média por década
w_decade = Window.partitionBy("decade")
w_all = Window.partitionBy()

desempenho_filmes = (
    desempenho_filmes
    .withColumn("avg_gross_decade", avg(col("gross")).over(w_decade))
    .withColumn("avg_gross_overall", avg(col("gross")).over(w_all))
    .withColumn("gross_above_decade_avg", col("gross") >= col("avg_gross_decade"))
    .withColumn("gross_above_overall_avg", col("gross") >= col("avg_gross_overall"))
)

# Seleção final (campos limpos e prontos para Tableau)
desempenho_filmes = desempenho_filmes.select(
    col("title_norm").alias("title"),
    "year",
    "decade",
    "gross",
    "avg_gross_decade",
    "avg_gross_overall",
    "gross_above_decade_avg",
    "gross_above_overall_avg",
    "won_any_oscar",
    "oscar_wins",
    "oscar_nominations",
    "oscar_categories",
    "is_adaptation",
    "authors",
    "book_titles",
    "n_authors",
    "n_books",
    "post_2000"
)

print("GOLD desempenho_filmes preview:")
desempenho_filmes.show(20, truncate=False)
print("Registos:", desempenho_filmes.count())


GOLD desempenho_filmes preview:
+-------------------------+----+------+-----------+--------------------+------------------+----------------------+-----------------------+-------------+----------+-----------------+-----------------------------------------------------------+-------------+-------+-----------+---------+-------+---------+
|title                    |year|decade|gross      |avg_gross_decade    |avg_gross_overall |gross_above_decade_avg|gross_above_overall_avg|won_any_oscar|oscar_wins|oscar_nominations|oscar_categories                                           |is_adaptation|authors|book_titles|n_authors|n_books|post_2000|
+-------------------------+----+------+-----------+--------------------+------------------+----------------------+-----------------------+-------------+----------+-----------------+-----------------------------------------------------------+-------------+-------+-----------+---------+-------+---------+
|the living daylights     |1987|1980  |5.1186196E7|1.865

In [8]:
# Criar tabela GOLD e gravar em Delta
spark.sql("DROP TABLE IF EXISTS gold.desempenho_filmes")

(
    desempenho_filmes.write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", True)
    .option("path", "hdfs://hdfs-nn:9000/warehouse/gold.db/desempenho_filmes")
    .saveAsTable("gold.desempenho_filmes")
)

print("Tabela gold.desempenho_filmes gravada com sucesso!")


Tabela gold.desempenho_filmes gravada com sucesso!


In [9]:
# Validação rápida
df_check = spark.table("gold.desempenho_filmes")
print("Registos na gold.desempenho_filmes:", df_check.count())
df_check.show(10, truncate=False)
print(df_check.columns)


Registos na gold.desempenho_filmes: 8144
+-------------------------+----+------+-----------+--------------------+------------------+----------------------+-----------------------+-------------+----------+-----------------+-----------------------------------------------------------+-------------+-------+-----------+---------+-------+---------+
|title                    |year|decade|gross      |avg_gross_decade    |avg_gross_overall |gross_above_decade_avg|gross_above_overall_avg|won_any_oscar|oscar_wins|oscar_nominations|oscar_categories                                           |is_adaptation|authors|book_titles|n_authors|n_books|post_2000|
+-------------------------+----+------+-----------+--------------------+------------------+----------------------+-----------------------+-------------+----------+-----------------+-----------------------------------------------------------+-------------+-------+-----------+---------+-------+---------+
|the living daylights     |1987|1980  |5.118619

In [14]:
from pyspark.sql.functions import concat_ws, col
from pyspark.sql.types import ArrayType

# Carregar a tabela Gold
df = spark.table("gold.desempenho_filmes")

# Identificar automaticamente colunas do tipo array
array_cols = [
    f.name for f in df.schema.fields
    if isinstance(f.dataType, ArrayType)
]

# Converter todas as colunas array para string (CSV-safe)
df_export = df
for c in array_cols:
    df_export = df_export.withColumn(c, concat_ws(", ", col(c)))

# Exportar para CSV (1 ficheiro)
df_export.coalesce(1) \
    .write \
    .mode("overwrite") \
    .option("header", "true") \
    .csv("/tmp/desempenho_filmes_csv")


In [1]:
!ls /tmp/desempenho_filmes_csv

part-00000-e3ea0ef3-819c-42a0-948c-93e13a9f7297-c000.csv  _SUCCESS


In [3]:
# Copiar o ficheiro CSV gerado pelo Spark
!cp /tmp/desempenho_filmes_csv/part-00000-e3ea0ef3-819c-42a0-948c-93e13a9f7297-c000.csv\
/home/jovyan/EDSTD/csv/gold/desempenho_filmes.csv

In [5]:
# Confirmar que o ficheiro existe e ver tamanho
!ls -lah /home/jovyan/EDSTD/csv/gold/desempenho_filmes.csv

-rw-r--r-- 1 jovyan users 1010K Dec 27 19:02 /home/jovyan/EDSTD/csv/gold/desempenho_filmes.csv


In [14]:
# Questoes analiticas:
# Ler a tabela Gold
from pyspark.sql.functions import avg, expr, countDistinct, round

df = spark.table("gold.desempenho_filmes")
df.printSchema()
df.show(5, truncate=False)


root
 |-- title: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- decade: integer (nullable = true)
 |-- gross: double (nullable = true)
 |-- avg_gross_decade: double (nullable = true)
 |-- avg_gross_overall: double (nullable = true)
 |-- gross_above_decade_avg: boolean (nullable = true)
 |-- gross_above_overall_avg: boolean (nullable = true)
 |-- won_any_oscar: boolean (nullable = true)
 |-- oscar_wins: integer (nullable = true)
 |-- oscar_nominations: integer (nullable = true)
 |-- oscar_categories: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- is_adaptation: boolean (nullable = true)
 |-- authors: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- book_titles: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- n_authors: long (nullable = true)
 |-- n_books: long (nullable = true)
 |-- post_2000: boolean (nullable = true)

+-------------------------+----+------+-----------+-----------

In [15]:
# Questão 1
# As adaptações de livros que venceram Óscares tiveram receitas de bilheteira mais altas do que as que não venceram?
q1 = (
    df
    .filter(col("is_adaptation") == True)
    .groupBy("won_any_oscar")
    .agg(
        round(avg("gross"), 2).alias("avg_gross"),
        expr("percentile_approx(gross, 0.5)").alias("median_gross"),
        countDistinct(expr("concat(title, year)")).alias("n_filmes")
    )
    .orderBy("won_any_oscar", ascending=False)
)

q1.show(truncate=False)



+-------------+--------------+------------+--------+
|won_any_oscar|avg_gross     |median_gross|n_filmes|
+-------------+--------------+------------+--------+
|true         |5.3177581892E8|6.09016565E8|13      |
|false        |3.6368388135E8|2.86801374E8|63      |
+-------------+--------------+------------+--------+



In [18]:
# Questão 2
# Quantas adaptações literárias receberam prémios Óscar em comparação com o total de adaptações lançadas?
total_adaptations = (
    df
    .filter(col("is_adaptation") == True)
    .selectExpr("concat(title, year) as film_id")
    .distinct()
    .count()
)

adaptations_with_oscar = (
    df
    .filter((col("is_adaptation") == True) & (col("won_any_oscar") == True))
    .selectExpr("concat(title, year) as film_id")
    .distinct()
    .count()
)

percentagem = __builtins__.round(
    (adaptations_with_oscar / total_adaptations) * 100,
    2
)

print("Total de adaptações:", total_adaptations)
print("Adaptações premiadas com Óscar:", adaptations_with_oscar)
print("Percentagem premiada:", percentagem, "%")



Total de adaptações: 76
Adaptações premiadas com Óscar: 13
Percentagem premiada: 17.11 %


In [20]:
# Questão 4
# Os filmes premiados com Óscares geram receitas de bilheteira mais elevadas do que os não premiados?
q4 = (
    df
    .groupBy("won_any_oscar")
    .agg(
        round(avg("gross"), 2).alias("avg_gross"),
        expr("percentile_approx(gross, 0.5)").alias("median_gross"),
        countDistinct(expr("concat(title, year)")).alias("n_filmes")
    )
    .orderBy("won_any_oscar", ascending=False)
)

q4.show(truncate=False)


+-------------+--------------+------------+--------+
|won_any_oscar|avg_gross     |median_gross|n_filmes|
+-------------+--------------+------------+--------+
|true         |2.3398386276E8|1.07968758E8|378     |
|false        |7.478573644E7 |2.867413E7  |7766    |
+-------------+--------------+------------+--------+



In [21]:
# Questão 5
# As adaptações lançadas depois do ano 2000 tiveram receitas mais altas do que as lançadas anteriormente?
q5 = (
    df
    .filter(col("is_adaptation") == True)
    .groupBy("post_2000")
    .agg(
        round(avg("gross"), 2).alias("avg_gross"),
        expr("percentile_approx(gross, 0.5)").alias("median_gross"),
        countDistinct(expr("concat(title, year)")).alias("n_filmes")
    )
    .orderBy("post_2000", ascending=False)
)

q5.show(truncate=False)


+---------+--------------+------------+--------+
|post_2000|avg_gross     |median_gross|n_filmes|
+---------+--------------+------------+--------+
|true     |4.2300544558E8|3.51266433E8|65      |
|false    |2.1180147345E8|1.00853753E8|11      |
+---------+--------------+------------+--------+



In [22]:
# Questão 7
# Quais são as adaptações literárias com maiores receitas de bilheteira?
q7 = (
    df
    .filter(col("is_adaptation") == True)
    .select(
        "title",
        "year",
        "gross",
        "authors",
        "book_titles",
        "won_any_oscar"
    )
    .orderBy(col("gross").desc())
)

q7.show(20, truncate=False)


+------------------------------------------------------------+----+-------------+------------------+-------------------------------------------+-------------+
|title                                                       |year|gross        |authors           |book_titles                                |won_any_oscar|
+------------------------------------------------------------+----+-------------+------------------+-------------------------------------------+-------------+
|the lord of the rings the return of the king                |2003|1.140697968E9|[jrr tolkien]     |[the return of the king]                   |true         |
|the hobbit an unexpected journey                            |2012|1.017003568E9|[jrr tolkien]     |[the hobbit]                               |false        |
|jurassic park                                               |1993|9.78167947E8 |[michael crichton]|[jurassic park]                            |true         |
|harry potter and the sorcerers stone         

In [23]:
# Questão 8
# Quais foram as décadas com mais filmes com sucesso nas bilheteiras?
q8 = (
    df
    .filter(col("gross_above_decade_avg") == True)
    .groupBy("decade")
    .agg(
        countDistinct(expr("concat(title, year)")).alias("n_filmes_sucesso"),
        round(avg("gross"), 2).alias("avg_gross_sucesso")
    )
    .orderBy("decade")
)

q8.show(truncate=False)


+------+----------------+-----------------+
|decade|n_filmes_sucesso|avg_gross_sucesso|
+------+----------------+-----------------+
|1980  |324             |5.175894786E7    |
|1990  |507             |1.0520278782E8   |
|2000  |549             |2.5802094792E8   |
|2010  |519             |4.3058652643E8   |
|2020  |197             |2.9634397914E8   |
+------+----------------+-----------------+

